# Cella 1 - Importazioni:

In [ ]:
# Importazioni
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import mysql.connector
from dotenv import load_dotenv

# Configurazione pandas per mostrare tutte le colonne
pd.set_option('display.max_columns', None)  # ✅ Mostra tutte le colonne
pd.set_option('display.width', None)        # ✅ Nessun limite larghezza
pd.set_option('display.max_colwidth', None) # ✅ Mostra testo completo

# Configurazione grafici
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# Cella 2 - Connessione e caricamento dati:

In [ ]:
# Carica variabili d'ambiente
load_dotenv()

# Connessione al database
conn = mysql.connector.connect(
    host=os.getenv("DB_HOST"),
    user=os.getenv("DB_USER"),
    password=os.getenv("DB_PASSWORD"),
    database=os.getenv("DB_NAME"),
    auth_plugin='mysql_native_password'
)

# Legge tutti i dati da raw_data
query = "SELECT * FROM raw_data"
df = pd.read_sql(query, conn)

conn.close()

print(f" Dataset caricato: {df.shape[0]} righe, {df.shape[1]} colonne")
print("\n Prime 5 righe:")
df.head()

# Cella 3 - Info generale sul dataset:

In [ ]:
# Informazioni generali
print("Informazioni sul dataset:\n")
df.info()

print("\n Statistiche descrittive:")
df.describe()

# Cella 4 - Distribuzione età:

In [ ]:
# Distribuzione età
plt.figure(figsize=(12, 5))

# Subplot 1: Istogramma
plt.subplot(1, 2, 1)
df['età'].dropna().astype(str).str.extract('(\d+)')[0].astype(int).hist(bins=30, edgecolor='black', alpha=0.7)
plt.xlabel('Età')
plt.ylabel('Frequenza')
plt.title('Distribuzione Età (Raw Data)')
plt.axvline(19, color='red', linestyle='--', label='Min età target (19)') 
plt.axvline(30, color='red', linestyle='--', label='Max età target (30)')
plt.legend()

# Subplot 2: Boxplot
plt.subplot(1, 2, 2)
età_clean = df['età'].dropna().astype(str).str.extract('(\d+)')[0].astype(int)
sns.boxplot(y=età_clean)
plt.ylabel('Età')
plt.title('Boxplot Età')

plt.tight_layout()
plt.show()

# Statistiche età
print(f"📈 Statistiche età:")
print(f"   - Min: {età_clean.min()}")
print(f"   - Max: {età_clean.max()}")
print(f"   - Media: {età_clean.mean():.2f}")
print(f"   - Mediana: {età_clean.median()}")
print(f"\n⚠️ Righe fuori range (< 19 o > 30): {((età_clean < 19) | (età_clean > 30)).sum()}")  # ✅ Cambiato

# Cella 5 - Distribuzione genere:

In [ ]:
# Distribuzione genere
plt.figure(figsize=(10, 5))

# Subplot 1: Barplot
plt.subplot(1, 2, 1)
df['genere'].value_counts().plot(kind='bar', edgecolor='black', alpha=0.7)
plt.xlabel('Genere')
plt.ylabel('Frequenza')
plt.title('Distribuzione Genere')
plt.xticks(rotation=45)

# Subplot 2: Pie chart
plt.subplot(1, 2, 2)
df['genere'].value_counts().plot(kind='pie', autopct='%1.1f%%', startangle=90)
plt.ylabel('')
plt.title('Proporzione Genere')

plt.tight_layout()
plt.show()

print("\n📊 Conteggio per genere:")
print(df['genere'].value_counts())

# Cella 6 - Distribuzione contesto:

In [ ]:
# Distribuzione contesto (studio/lavoro/entrambi)
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
df['contesto'].value_counts().plot(kind='bar', edgecolor='black', alpha=0.7, color='skyblue')
plt.xlabel('Contesto')
plt.ylabel('Frequenza')
plt.title('Distribuzione Contesto')
plt.xticks(rotation=45)

plt.subplot(1, 2, 2)
df['contesto'].value_counts().plot(kind='pie', autopct='%1.1f%%', startangle=90, colors=sns.color_palette("pastel"))
plt.ylabel('')
plt.title('Proporzione Contesto')

plt.tight_layout()
plt.show()

print("\n📊 Conteggio per contesto:")
print(df['contesto'].value_counts())

# Cella 7 - Distribuzione area geografica:

In [ ]:
# Distribuzione area geografica
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
df['area_geo'].value_counts().plot(kind='barh', edgecolor='black', alpha=0.7, color='coral')
plt.xlabel('Frequenza')
plt.ylabel('Area Geografica')
plt.title('Distribuzione Area Geografica')

plt.subplot(1, 2, 2)
df['area_geo'].value_counts().plot(kind='pie', autopct='%1.1f%%', startangle=90)
plt.ylabel('')
plt.title('Proporzione Area Geografica')

plt.tight_layout()
plt.show()

print("\n📊 Conteggio per area geografica:")
print(df['area_geo'].value_counts())

# Cella 8 - Distribuzione Items (scala Likert):

In [ ]:
# Distribuzione delle risposte agli Item (Item_1 a Item_29)
item_cols = [col for col in df.columns if col.startswith('Item_') and col != 'Item_30']

plt.figure(figsize=(15, 10))

for i, col in enumerate(item_cols[:12], 1):  # Prime 12 domande
    plt.subplot(3, 4, i)
    df[col].value_counts().sort_index().plot(kind='bar', edgecolor='black', alpha=0.7)
    plt.xlabel('Punteggio')
    plt.ylabel('Frequenza')
    plt.title(f'{col}')
    plt.xticks(rotation=0)

plt.tight_layout()
plt.show()

# Statistiche degli Item
print("\n📊 Statistiche Item (scala 1-5):")
print(df[item_cols].describe())

# Cella 9 - Heatmap correlazioni Items:

In [ ]:
# Heatmap delle correlazioni tra Item
plt.figure(figsize=(16, 14))

correlazioni = df[item_cols].corr()

sns.heatmap(correlazioni, annot=False, cmap='coolwarm', center=0, 
            square=True, linewidths=0.5, cbar_kws={"shrink": 0.8})

plt.title('Matrice di Correlazione tra Item (Ansia da Prestazione)', fontsize=16)
plt.tight_layout()
plt.show()

# Correlazioni più alte
print("\n🔥 Correlazioni più forti (> 0.7):")
mask = np.triu(np.ones_like(correlazioni, dtype=bool))
corr_flat = correlazioni.mask(mask).stack()
print(corr_flat[corr_flat > 0.7].sort_values(ascending=False).head(10))

# Cella 10 - Valori mancanti:

In [ ]:
# Analisi valori mancanti
missing = df.isnull().sum()
missing_pct = (missing / len(df)) * 100

missing_df = pd.DataFrame({
    'Colonna': missing.index,
    'Valori Mancanti': missing.values,
    'Percentuale': missing_pct.values
})

missing_df = missing_df[missing_df['Valori Mancanti'] > 0].sort_values('Valori Mancanti', ascending=False)

if len(missing_df) > 0:
    plt.figure(figsize=(12, 6))
    plt.barh(missing_df['Colonna'], missing_df['Percentuale'], edgecolor='black', alpha=0.7, color='salmon')
    plt.xlabel('Percentuale Valori Mancanti (%)')
    plt.title('Valori Mancanti per Colonna')
    plt.tight_layout()
    plt.show()
    
    print("\n⚠️ Colonne con valori mancanti:")
    print(missing_df)
else:
    print("\n✅ Nessun valore mancante nel dataset!")

# Cella 11 - Summary report (correzione):

In [ ]:
# Summary finale
print("=" * 60)
print("📊 SUMMARY ANALISI ESPLORATIVA - RAW DATA")
print("=" * 60)
print(f"\n✅ Totale risposte: {len(df)}")
print(f"\n📈 Età:")
print(f"   - Range: {età_clean.min()} - {età_clean.max()}")
print(f"   - Media: {età_clean.mean():.1f} anni")
print(f"   - Righe da escludere (età < 19 o > 30): {((età_clean < 19) | (età_clean > 30)).sum()}")  # ✅ Cambiato
print(f"\n👥 Genere:")
print(df['genere'].value_counts())
print(f"\n🏢 Contesto:")
print(df['contesto'].value_counts())
print(f"\n🗺️ Area Geografica:")
print(df['area_geo'].value_counts())
print("\n" + "=" * 60)